In [29]:
  
__metaclass__=type
import random
import csv
import matplotlib.pyplot as plt
%matplotlib inline

#define a class called 'Customer'
class Customer:
    def __init__(self,arrival_time,service_start_time,service_time, item):
        self.arrival_time=arrival_time
        self.service_start_time=service_start_time
        self.service_time=service_time
        self.service_end_time=self.service_start_time+self.service_time
        self.wait=self.service_start_time-self.arrival_time
        self.item = item
        
class Cashier:
    def __init__(self,serviceRate):
        self.serviceRate = serviceRate
        self.available = True
        self.ServiceEnd= -1
    def checkAvailability(self,currentTime):
        if(currentTime>self.ServiceEnd):
            self.available =True
        return self.available
    def serve(self,serviceStartTime,item):
        self.ServiceEnd = serviceStartTime + neg_exp(self.serviceRate)
        self.available = False
        
        
class Store:
    def __init__(self,Name,Num_Customer,Mean_Delay,Mean_service_Delay,Num_Manager_resquest, stock):
        self.Name=Name
        self.Num_Customer=Num_Customer
        self.Mean_Delay=Mean_Delay
        self.Mean_service_Delay=Mean_service_Delay
        self.Num_Manager_resquest=Num_Manager_resquest
        self.stock = stock

#Store In a simulation since you only make the store at the end of its simulation
class StoreInProgress:
    def __init__(self,stock_max,arrival_rate,cashiers):
        self.stock_max = stock_max
        self.stock_current = stock_max
        self.stock_sold = [0,0,0]
        self.arrival_rate = arrival_rate
        self.Cashiers= cashiers
        self.customers= []
        self.customerWait = []
        self.Num_Manager_resquest=0
        self.nextEvent=0
    def replenish(self,item, amount):
        self.stock_current[item] = self.stock_current[item] + amount
    
        
class Shelf:
    def __init__(self,):
        self.Name=Name
        self.Num_Customer=Num_Customer
        self.Mean_Delay=Mean_Delay
        self.Mean_service_Delay=Mean_service_Delay
        self.Num_Manager_resquest=Num_Manager_resquest
        
        
#a simple function to sample from negative exponential
def neg_exp(lambd):
    return random.expovariate(lambd)

def item_choose():
    return random.randint(0, 2)  # Random integer x, 1 <= x < 10

def Manager_Queue_Delay():
    return random.randint(10, 60)  # Random integer x, 1 <= x < 10


def StoreSim(lambd=False,mu=False,simulation_time=False):
    """
    This is the main function to call to simulate a store.
    """
    max_Stock = [25,25,25]
    max_Stock2 = [15,25,17]
    max_Stock3 = [21,13,32]
    print "So is this function even working?"
    
    #If parameters are not input prompt
    if not lambd:
        lambd=input('Inter arrival rate: ')
        lambd=int(lambd)
    if not mu:
        mu=input('Service rate: ')
        mu=int(mu)
    if not simulation_time:
        simulation_time=input('Total simulation time: ')
        simulation_time=int(simulation_time)

    #Initialise clock
    t=0

    #Initialise empty list to hold all data
    Customers=[]
    CustomerWait=[]
    Cashiers = []
    Cashiers2 = []
    Cashiers3 = []
    
    StoreEvents=[]
    
    
    Cashiers.append(Cashier(neg_exp(5)))
    Cashiers.append(Cashier(neg_exp(7)))
    Cashiers.append(Cashier(neg_exp(3)))
    
    Cashiers2.append(Cashier(neg_exp(7)))
    Cashiers2.append(Cashier(neg_exp(3)))
    
    Cashiers3.append(Cashier(neg_exp(7)))
    Cashiers3.append(Cashier(neg_exp(3)))
    Cashiers3.append(Cashier(neg_exp(7)))
    Cashiers3.append(Cashier(neg_exp(3)))
    
   
    
    Num_Manager_resquest = 0
    Store1 = StoreInProgress(max_Stock,neg_exp(2), Cashiers)
    Store2 = StoreInProgress(max_Stock2,neg_exp(2), Cashiers2)
    Store3 = StoreInProgress(max_Stock3,neg_exp(2), Cashiers3)
    
    stores = []
    stores.append(Store1)
    stores.append(Store2)
    stores.append(Store3)
    print "well everything is initialized so lets just look at the times quickly"
    for store in stores:
        store.nextEvent = neg_exp(store.arrival_rate)
        print str(store.nextEvent)
        StoreEvents.append(store.nextEvent)
    
    
#----------------------------------
#The actual simulation happens here:
    while t<simulation_time:
        print "The simulation is running"
        item = item_choose()
        #calculate arrival date and service time for new customer
        next_arrival = StoreEvents[0]
        storeNum =0
        i =0
        for storeEvent in StoreEvents:
            if(storeEvent<next_arrival):
                next_arrival= storeEvent
                storeNum=i
            i= i+1
        print "Next Arrival " + str(next_arrival) + " Store" + str(storeNum)
        
        #if len(stores[storeNum].customers)==0:
        service_start_time = next_arrival
        busy = False
        i = 0
        #check if the item is in stock
        if stores[storeNum].stock_current[item]> 0:
            service_time=0
            available = False
            #look for an available cashier
            for cash in stores[storeNum].Cashiers:
                if(cash.checkAvailability(service_start_time)):
                    cash.serve(service_start_time,item)
                    service_time=cash.ServiceEnd - service_start_time
                    available = True
                    break
                
            #if no cashier is available wait for the next available Cashier
            if(not available):
                minTime = stores[storeNum].Cashiers[0].ServiceEnd
                chold = 0
                for cash in range(len(stores[storeNum].Cashiers)):
                    if(stores[storeNum].Cashiers[cash].ServiceEnd< minTime):
                        minTime = stores[storeNum].Cashiers[cash].ServiceEnd
                        chold = cash
                service_start_time = minTime
                stores[storeNum].Cashiers[chold].serve(service_start_time,item)
                service_time = stores[storeNum].Cashiers[chold].ServiceEnd - service_start_time
            #Update Stock
            stores[storeNum].stock_current[item] = Store1.stock_current[item] - 1
            stores[storeNum].stock_sold[item] = Store1.stock_sold[item] + 1
            stores[storeNum].nextEvent = next_arrival + neg_exp(stores[storeNum].arrival_rate);
            StoreEvents[storeNum] = stores[storeNum].nextEvent
        else:
            service_time=neg_exp(mu)+Manager_Queue_Delay()
            stores[storeNum].Num_Manager_resquest = stores[storeNum].Num_Manager_resquest+1
            #replenish stock if out
                
            stores[storeNum].replenish(item,10)
        """
        else:
            arrival_time+=neg_exp(lambd)
            service_start_time=max(arrival_time,Customers[-1].service_end_time)
            if Store1.stock_current[item]> 0:
                service_time = 0
                available = False
                #look for an available cashier
                for cash in Cashiers:
                    if(cash.checkAvailability(service_start_time)):
                        cash.serve(service_start_time,item)
                        service_time=cash.ServiceEnd - service_start_time
                        available = True
                        break
                
                #if no cashier is available wait for the next available Cashier
                if(not available):
                    minTime = Cashiers[0].ServiceEnd
                    chold = 0
                    for cash in range(len(Cashiers)):
                        if(Cashiers[cash].ServiceEnd< minTime):
                            minTime = Cashiers[cash].ServiceEnd
                            chold = cash
                    Cashiers[chold].serve(minTime,item)
                    service_time = Cashiers[chold].ServiceEnd - service_start_time
                Store1.stock_current[item] = Store1.stock_current[item] - 1
                Store1.stock_sold[item] = Store1.stock_sold[item] + 1
            else:
                service_time=neg_exp(mu)+Manager_Queue_Delay()
                Num_Manager_resquest = Num_Manager_resquest+1
                Store1.replenish(item,10)
        """
        #create new customer and add him to Customers list
        stores[storeNum].customers.append(Customer(next_arrival,service_start_time,service_time,item))
        
        #increment clock till next end of service
        t=next_arrival
#----------------------------------
    i2=0
    stats = []
    for store in stores:
        print "Customer Length " + str(len(store.customers))
        #calculate summary statistics
        Waits=[a.wait for a in store.customers]
        Mean_Wait=sum(Waits)/len(Waits)

        Total_Times=[a.wait+a.service_time for a in store.customers]
        Mean_Delay=sum(Total_Times)/len(Total_Times)

        Service_Times=[a.service_time for a in store.customers]
        Mean_Service_Delay=sum(Service_Times)/len(Service_Times)
        stats.append(Store('store'+str(i2),len(store.customers),Mean_Delay,Mean_Service_Delay,store.Num_Manager_resquest,store.stock_current))
        
    Waits=[a.wait for a in stores[0].customers]
    Mean_Wait=sum(Waits)/len(Waits)

    Total_Times=[a.wait+a.service_time for a in stores[0].customers]
    Mean_Delay=sum(Total_Times)/len(Total_Times)

    Service_Times=[a.service_time for a in stores[0].customers]
    Mean_Service_Delay=sum(Service_Times)/len(Service_Times)
    print "Store output"
    return Store('store'+str(0),len(stores[0].customers),Mean_Delay,Mean_Service_Delay,stores[0].Num_Manager_resquest,stores[0].stock_current)

In [30]:

        
def NetworkSim(num_store=False,simulation_time=False):
    """
    This is the main function to call to simulate the network.
    """

    #If parameters are not input prompt user
    if not num_store:
        num_store=input('Enter the number of Stores: ')
    
    if not simulation_time:
        simulation_time=input('Total simulation time: ')
        

    #Initialise empty list to collect store info
    Net_Stores=[]
    
    for i in range(num_store):
        Lambd = neg_exp(1)
        Mu = neg_exp(2)
        Net_Stores.append(StoreSim(Lambd, Mu, simulation_time))
    
    #map of medicine made
    item_map = ['Tynenol', 'Cold Medicine', 'Vitamins']
    
    print ("         Summary results:        ")
    print("")
    print('StoreName   Num_Customer   Mean_Delay    Mean_service_Delay    Num_Manager_resquest    '  + item_map[0] + '     ' +  item_map[1]+ '     '  + item_map[2])
    j=0
    
    
    for store in Net_Stores:
        j = j+1
        print("%s %d         %d              %d              %d                    %d                   %d                %d            %d"% 
              (store.Name,j,store.Num_Customer,store.Mean_Delay,store.Mean_service_Delay,store.Num_Manager_resquest,store.stock[0],store.stock[1],store.stock[2])) 
        print("")
        
        #output summary statistics to screen
        #print ("")
        #print ("Store:", store.Name)
        #print ("Number of customers: ",store.Num_Customer)
        #print ("Mean Service Time: ",store.Mean_service_Delay)
        #print ("Mean Time in System: ",store.Mean_Delay)
        #print ("Number of Manager request: ",store.Num_Manager_resquest)
        #print ("")
        


In [31]:
NetworkSim(1, 40)
"Well?"

So is this function even working?
well everything is initialized so lets just look at the times quickly
0.854222185504
0.0444760306168
3.49690908604
The simulation is running
Next Arrival 0.0444760306168 Store1
The simulation is running
Next Arrival 0.176029738422 Store1
The simulation is running
Next Arrival 0.527033769127 Store1
The simulation is running
Next Arrival 0.854222185504 Store0
The simulation is running
Next Arrival 0.894656826419 Store0
The simulation is running
Next Arrival 2.26827913582 Store1
The simulation is running
Next Arrival 2.83476305084 Store1
The simulation is running
Next Arrival 3.49690908604 Store2
The simulation is running
Next Arrival 3.64661441541 Store2
The simulation is running
Next Arrival 3.93503016365 Store1
The simulation is running
Next Arrival 4.04508581182 Store1
The simulation is running
Next Arrival 4.36115965128 Store2
The simulation is running
Next Arrival 4.83604704802 Store1
The simulation is running
Next Arrival 5.05599614495 Store1
The s

'Well?'

In [32]:
def main():
    # open input and out files
    with open('mm1.csv', 'r') as infile:
    with open('mmm2.csv', 'w') as outfile:
    
    #specify the number of event for the timing function
    
    num_events =2
     
    #read input parameters
    infile.read()
    
    
    
    while num_cust_delay<num_delay_required:
        #determine the next evet
        
        initialize()
        
        update_time_avg_stats()
        
        # invoke the appropriate event function
        switch(next_time_type):
            case 1:
                arrival()
                break
            case 2:
                depart()
                break
        report()
    
    
def timing:
    
    min_time_next_event = 1.0e+29
    next_event_type = 0
    
    #Determine the event type of the next event to occur
    
    for i in range(num_events):
        if time_next_event[i+1]< min_time_next_event:
            min_time_next_event = time_next_event[i+1]
            next_event_type = i+1
            
    # check to see weither the event list is empty
    if next_event_type == 0:
        
    
    
    
def initialize:
    
    
    
def arrival:
    
    
def depart:
    
    
def report:
    
    
def update_time_avg_stats:
    
    
    
def expon:
    


IndentationError: expected an indented block (<ipython-input-32-cbb7e169f58c>, line 4)